In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('drive/My Drive/GAN_in_art-museum')

# !pip install -r requirements.txt

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [15]:
from keras.datasets import mnist
from models.AE2 import Autoencoder

# 실행 매개변수
SECTION = 'vae'
RUN_ID = '0001'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir('run')
    os.mkdir('run/{}' .format(SECTION))
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

MODE =  'build' #'load' #

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [4]:
import numpy as np

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = np.expand_dims(x_test, axis = -1)

In [9]:
AE = Autoencoder(
    input_dim = (28, 28, 1),
    encoder_conv_filters = [32, 64, 64, 64],
    encoder_conv_kernel_size = [3, 3, 3, 3],
    encoder_conv_strides = [1, 2, 2, 1],
    decoder_conv_t_filters = [64, 64, 32, 1],
    decoder_conv_t_kernel_size = [3,3,3,3],
    decoder_conv_t_strides = [1,2,2,1],
    z_dim = 2
)

if MODE == 'build':
  AE.save(RUN_FOLDER)
else:
  AE.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [10]:
AE.encoder.summary()

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 28, 28, 1)         0         
_________________________________________________________________
encoder_conv_0 (Conv2D)      (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 28, 28, 32)        0         
_________________________________________________________________
encoder_conv_1 (Conv2D)      (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
encoder_conv_2 (Conv2D)      (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 7, 7, 64)          0  

In [11]:
AE.decoder.summary()

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 2)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_3 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 14, 14, 64)        0  

In [14]:
# training

LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0

AE.compile(LEARNING_RATE)

In [ ]:
history=AE.train(
    x_train,
    batch_size = BATCH_SIZE,
    epochs = 200,
    run_folder = RUN_FOLDER,
    initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
  736/60000 [..............................] - ETA: 9:12 - loss: 0.1838 

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.176144). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


60000/60000 [==============================] - 19s 324us/step - loss: 0.0545

Epoch 00001: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 2/200
60000/60000 [==============================] - 13s 214us/step - loss: 0.0459

Epoch 00002: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 3/200
60000/60000 [==============================] - 13s 216us/step - loss: 0.0441

Epoch 00003: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 4/200
60000/60000 [==============================] - 13s 216us/step - loss: 0.0431

Epoch 00004: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 5/200
60000/60000 [==============================] - 13s 216us/step - loss: 0.0424

Epoch 00005: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 6/200
60000/60000 [==============================] - 13s 216us/step - loss: 0.0418

Epoch 00006: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 7/200
60000/60000 [==============================] - 13s 2

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.150069). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


60000/60000 [==============================] - 13s 216us/step - loss: 0.0402

Epoch 00012: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 13/200
60000/60000 [==============================] - 13s 217us/step - loss: 0.0400

Epoch 00013: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 14/200
60000/60000 [==============================] - 13s 220us/step - loss: 0.0399

Epoch 00014: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 15/200
60000/60000 [==============================] - 13s 214us/step - loss: 0.0398

Epoch 00015: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 16/200
60000/60000 [==============================] - 13s 215us/step - loss: 0.0396

Epoch 00016: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 17/200
60000/60000 [==============================] - 13s 213us/step - loss: 0.0395

Epoch 00017: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 18/200
60000/60000 [==============================] -

...

60000/60000 [==============================] - 13s 218us/step - loss: 0.0381

Epoch 00037: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 38/200
60000/60000 [==============================] - 13s 213us/step - loss: 0.0380

Epoch 00038: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 39/200
60000/60000 [==============================] - 13s 214us/step - loss: 0.0380

Epoch 00039: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 40/200
60000/60000 [==============================] - 13s 214us/step - loss: 0.0379

Epoch 00040: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 41/200
60000/60000 [==============================] - 13s 216us/step - loss: 0.0379

Epoch 00041: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 42/200
60000/60000 [==============================] - 13s 215us/step - loss: 0.0378

Epoch 00042: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 43/200
60000/60000 [==============================] -

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.138192). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


60000/60000 [==============================] - 13s 215us/step - loss: 0.0375

Epoch 00051: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 52/200
60000/60000 [==============================] - 13s 211us/step - loss: 0.0375

Epoch 00052: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 53/200
60000/60000 [==============================] - 13s 213us/step - loss: 0.0374

Epoch 00053: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 54/200
60000/60000 [==============================] - 13s 211us/step - loss: 0.0374

Epoch 00054: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 55/200
60000/60000 [==============================] - 13s 221us/step - loss: 0.0374

Epoch 00055: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 56/200
60000/60000 [==============================] - 13s 215us/step - loss: 0.0373

Epoch 00056: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 57/200
60000/60000 [==============================] -

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.159832). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


60000/60000 [==============================] - 13s 214us/step - loss: 0.0369

Epoch 00072: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 73/200
60000/60000 [==============================] - 13s 215us/step - loss: 0.0369

Epoch 00073: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 74/200
60000/60000 [==============================] - 13s 212us/step - loss: 0.0369

Epoch 00074: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 75/200
60000/60000 [==============================] - 13s 217us/step - loss: 0.0368

Epoch 00075: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 76/200
60000/60000 [==============================] - 13s 214us/step - loss: 0.0369

Epoch 00076: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 77/200
60000/60000 [==============================] - 13s 213us/step - loss: 0.0368

Epoch 00077: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 78/200
60000/60000 [==============================] -

In [ ]:
history_dict = history.history
loss = history_dict['loss']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'b', label = 'Loss')

plt.rcParams['figure.figsize'] = (18, 8)
plt.rcParams['line.linewidth'] = 1.5
plt.rcParmas['axes.gird'] = True
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()